based on [IfcOpenShell and PythonOCC acadmey example](https://academy.ifcopenshell.org/posts/using-ifcopenshell-and-pythonocc-to-construct-new-geometry/)

Almost nothing here worked. The code was heavilty edited with examples from /ref-gits/pythonocc-demos/jupyter_notebooks

In [1]:
import OCC
import ifcopenshell, ifcopenshell.geom

In [2]:
# Open the IFC file using IfcOpenShell
ifc = ifcopenshell.open("../data/IfcOpenHouse_IFC4.ifc")

# Displaying The House

In [3]:
#THIS IS A DIRECT COPY.DOESNT WORK. IT APEARS MODULE ifcopenshell version 0.6 is problematic?

# # Specify to return pythonOCC shapes from ifcopenshell.geom.create_shape()
# settings = ifcopenshell.geom.settings()
# settings.set(settings.USE_PYTHON_OPENCASCADE, True)

# # Initialize a graphical display window
# occ_display = ifcopenshell.geom.utils.initialize_display()

# # Open the IFC file using IfcOpenShell
# ifc_file = ifcopenshell.open("IfcOpenHouse.ifc")
# # Display the geometrical contents of the file using Python OpenCascade
# products = ifc_file.by_type("IfcProduct")
# for product in products:
#     if product.is_a("IfcOpeningElement"): continue
#     if product.Representation:
#         shape = ifcopenshell.geom.create_shape(settings, product).geometry
#         display_shape = ifcopenshell.geom.utils.display_shape(shape)
#         if product.is_a("IfcPlate"):
#             # Plates are the transparent parts of the window assembly
#             # in the IfcOpenHouse model
#             ifcopenshell.geom.utils.set_shape_transparency(display_shape, 0.8)

In [4]:
# Using what we know from "helloWorld.ipynb" in this src folder we can display an image, but it crashes our kernel

# from OCC.Display.SimpleGui import init_display
# occ_display, start_display, add_menu, add_function_to_menu = init_display()

# # Specify to return pythonOCC shapes from ifcopenshell.geom.create_shape()
# settings = ifcopenshell.geom.settings()
# settings.set(settings.USE_PYTHON_OPENCASCADE, True)

# #occ_display = ifcopenshell.geom.utils.initialize_display()
# products = ifc.by_type("IfcProduct")
# for product in products:
#     if product.is_a("IfcOpeningElement"): continue
#     if product.Representation:
#         shape = ifcopenshell.geom.create_shape(settings, product).geometry
#         display_shape = occ_display.DisplayShape(shape,update=True)
#         #if product.is_a("IfcPlate"):
#             # Plates are the transparent parts of the window assembly
#             # in the IfcOpenHouse model
#             #ifcopenshell.geom.utils.set_shape_transparency(display_shape, 0.8)

## Lets use the jupyter renderer to display it 'inline'

In [5]:
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer, format_color
my_renderer = JupyterRenderer(size=(500, 350))

# Specify to return pythonOCC shapes from ifcopenshell.geom.create_shape()
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)

In [6]:
products = ifc.by_type("IfcProduct") # traverse all IfcProducts

for product in products:
    if product.is_a("IfcOpeningElement"): continue
    if product.Representation is not None:  # some IfcProducts don't have any 3d representation
        pdct_shape = ifcopenshell.geom.create_shape(settings, inst=product)

        if product.is_a("IfcPlate"):
            trans = True
        else:
            trans = False

        r,g,b,alpha = pdct_shape.styles[0] # the shape color
        color = format_color(int(abs(r)*255), int(abs(g)*255), int(abs(b)*255))
        # below, the pdct_shape.geometry is a TopoDS_Shape, i.e. can be rendered using
        # any renderer (threejs, x3dom, jupyter, qt5 based etc.)
        my_renderer.DisplayShape(pdct_shape.geometry, shape_color = color, transparency=trans, opacity=alpha)

In [7]:
my_renderer

/home/alon/LocalInstalls/miniconda3/envs/pyoccenv/lib/python3.7/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


#  Filtering Elements and getting bounding box of walls only

helpful: https://programtalk.com/python-examples/OCC.BRepBndLib.brepbndlib_Add/

In [77]:
my_renderer.EraseAll()

# Get a list of all walls in the file
walls = ifc.by_type("IfcWall")

# Create a list of wall representation shapes
# and compute the bounding box of these shapes

wall_shapes = [] #note, this will be used laterz
bbox = OCC.Core.Bnd.Bnd_Box()
for wall in walls:
    shape = ifcopenshell.geom.create_shape(settings, wall).geometry
    
    wall_shapes.append((wall, shape))
    OCC.Core.BRepBndLib.brepbndlib_Add(shape,bbox)  
    
    my_renderer.DisplayShape(shape, render_edges=False)
    
#Calculate the center/average of the bounding box
bounding_box_center = ifcopenshell.geom.utils.get_bounding_box_center(bbox)

# Found this by accident, will it be helpfull?
# OCC.Display.OCCViewer.Display3d.GetImageData

print("Center = {}".format(bounding_box_center.Coord()))

Center = (0.0, 2.5000000000000004, 2.75)


In [78]:
my_renderer

# Extracting Faces and Manipulation on Normals

note: in open cascade and most other geomtrey kernels there is a distinction between geomtrey and topoly

https://opencascade.blogspot.com/2009/02/topology-and-geometry-in-open-cascade.html

reading it, I can summaraize as such:
    topolgy is the complex structure (face,shell,solid...) that uses simple geomtries (point, plane ...)

In [79]:
#note.. over here the reference has us call a module called "OCC.Utils" which we dont have. Lets investigate:
import site; print(''.join(site.getsitepackages())) #this brings us to where all the packages in our venv are installed
#nope nothing here...
#this comes from a git https://github.com/tpaviot/pythonocc-utils. (installed with pip)


/home/alon/LocalInstalls/miniconda3/envs/pyoccenv/lib/python3.7/site-packages


In [124]:
my_renderer.EraseAll()

import OCCUtils

# Now create halfspace solids from the inner faces of the wall
halfspaces = []
outside_faces = []
for wall, shape in wall_shapes:
    
    topo = OCCUtils.Topo(shape) #from https://pythonocc-doc.readthedocs.io/en/latest/geom_intro/
    for face in topo.faces():
        surf = OCC.Core.BRep.BRep_Tool.Surface(face)
        
        ## methods that dont exist anymore
        #obj = surf.GetObject() 
        #assert obj.DynamicType().GetObject().Name() == "Geom_Plane"
        
        plane = OCC.Core.Geom.Handle_Geom_Plane_DownCast(surf)
        
        if plane.Axis().Direction().Z() == 0:
            face_bbox = OCC.Core.Bnd.Bnd_Box()
            OCC.Core.BRepBndLib.brepbndlib_Add(face, face_bbox)
            face_center = ifcopenshell.geom.utils.get_bounding_box_center(face_bbox).XYZ()
            
            face_normal = plane.Axis().Direction().XYZ()
            vector_towards_center = bounding_box_center.XYZ() - face_center
            vector_towards_center.Normalize()
            
            dot = vector_towards_center.Dot(face_normal)

            #normals of shapes go inside the shapes. That is to say that walls on the outside of the house will have normals that point into it's center.
            
            if dot > 0.8:
                my_renderer.DisplayShape(face,shape_color='#2EA320') #green 

                face_plane = plane.Pln()
                new_face = OCC.Core.BRepBuilderAPI.BRepBuilderAPI_MakeFace(face_plane).Face()
                halfspace = OCC.Core.BRepPrimAPI.BRepPrimAPI_MakeHalfSpace(
                    new_face, bounding_box_center).Solid()
                halfspaces.append(halfspace)

                outside_faces.append(face)
            else:

                my_renderer.DisplayShape(face,shape_color='#FF5733') #red


In [125]:
my_renderer

# Creating Halfspace Solids from the bottom faces of the roofs

In [99]:
my_renderer.EraseAll()

# Create halfspace solids from the bottom faces of the roofs

roofs = ifc.by_type("IfcSlab") #I went to the IFC file and tried to figure out how the fuck it can be called, because ifcRoof was no good
for roof in roofs:
    roof.Representation is not None
    shape = ifcopenshell.geom.create_shape(settings, roof).geometry
    
    topo = OCCUtils.Topo(shape)
    for face in topo.faces():
        surf = OCC.Core.BRep.BRep_Tool.Surface(face)
        plane = OCC.Core.Geom.Handle_Geom_Plane_DownCast(surf)

        if plane.Axis().Direction().Z() > 0.5:
            
            my_renderer.DisplayShape(face,shape_color='#FF5733') #red

            face_plane = plane.Pln()
            new_face = OCC.Core.BRepBuilderAPI.BRepBuilderAPI_MakeFace(face_plane).Face()
            halfspace = OCC.Core.BRepPrimAPI.BRepPrimAPI_MakeHalfSpace(
                new_face, bounding_box_center).Solid()
            halfspaces.append(halfspace)

        else:

            my_renderer.DisplayShape(face,shape_color='#2EA320') #green

In [100]:
my_renderer

/home/alon/LocalInstalls/miniconda3/envs/pyoccenv/lib/python3.7/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


# Using Booleanic operations to compute the volume of the house

In [101]:
my_renderer.EraseAll()
from OCC.Core import BRepAlgoAPI #btw BRep stands for boundry representation

a = 10

# Create an initial box from which to cut the halfspaces
common_shape = OCC.Core.BRepPrimAPI.BRepPrimAPI_MakeBox(
    OCC.Core.gp.gp_Pnt(-a, -a, 0),
    OCC.Core.gp.gp_Pnt(a, a, a)).Solid()
for halfspace in halfspaces:    
     common_shape = BRepAlgoAPI.BRepAlgoAPI_Common(common_shape, halfspace).Shape()

my_renderer.DisplayShape(common_shape,shape_color='#2EA320') #green

# Calculate the volume properties of the resulting space shape
props = OCC.Core.GProp.GProp_GProps()
OCC.Core.BRepGProp.brepgprop_VolumeProperties(common_shape, props)
print("Space volume: {} cubic meter".format(props.Mass()))

Space volume: 195.28838399999998 cubic meter


In [102]:
my_renderer

# Distinguishing walls Facing X-East

In [137]:
my_renderer.EraseAll()

for face in outside_faces:
        surf = OCC.Core.BRep.BRep_Tool.Surface(face)
        plane = OCC.Core.Geom.Handle_Geom_Plane_DownCast(surf)

        if abs(plane.Axis().Direction().X()) > 0:
                my_renderer.DisplayShape(face,shape_color='#2EA320') #green
        else:
                my_renderer.DisplayShape(face,shape_color='#FF5733') #red

In [138]:
my_renderer